### Root Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 28  # native word end index

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/4-Root Analysis/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [8]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.lower().capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abluka,blockade
2,absorbe,absorb
3,absürt,absurd
4,açelya,azalea
...,...,...
1773,zebra,zebra
1774,zikzak,zigzag
1775,zombi,zombie
1776,zooloji,zoology


In [10]:
# Option
if prefix_suffix:
    df_prefix_suffix_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
    df_prefix_suffix_select = df_prefix_suffix_select.loc[:,["search_word","word"]]
    df_prefix_suffix_select.rename(columns={"search_word":"dict_entry_main"}, inplace=True)
    df_pair = pd.merge(df_pair,df_prefix_suffix_select, how="inner", on="dict_entry_main")
    df_pair.drop_duplicates(inplace=True)
    df_pair.reset_index(drop=True, inplace=True)
    df_pair = df_pair.loc[:,["word",f"{lang_pair.lower()}_word"]]
    df_pair.rename(columns={"word":"dict_entry_main"}, inplace=True)
else:
    pass

if etymology_word:
    df_pair
else:
    df_pair = df_pair.head(0)
    
df_pair

,dict_entry_main,english_word


In [11]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]
#disable_list = []

In [12]:
word_select = df_word["word"].values.tolist()
words = df_pair["dict_entry_main"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [13]:
word_list = list((word_select_set.union(words_set)).difference(disable_word_set))
len(word_list)

200

In [14]:
df_word_result = pd.DataFrame(word_list, columns=["word"])
df_word_result

,word
0,burada
1,bak
2,ben
3,olduğunu
4,biliyorum
...,...
195,git
196,önce
197,iyi
198,olarak


In [15]:
df_word_result = strip(df_word_result)
df_word_result

,word
0,burada
1,bak
2,ben
3,olduğunu
4,biliyorum
...,...
195,git
196,önce
197,iyi
198,olarak


In [16]:
df_lemma_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Data/{lang_folder.capitalize()}/Turkish_100000_Word_All_Lemma_Result.xlsx")
df_lemma_all

,word,lemma.atis,lemma.boun,lemma.framenet,lemma.imst,lemma,lemma.kenet,lemma.penn,lemma.spacylookup,lemma.tourism
0,bir,bir,bir,bir,bir,bir,bir,bir,bir,bir
1,bu,bu,bu,bu,bu,bu,bu,bu,bu,bu
2,ne,ne,ne,ne,ne,ne,ne,ne,ne,ne
3,ve,ve,ve,ve,ve,ve,ve,ve,ve,ve
4,için,için,için,için,için,da,iç,da,için,için
...,...,...,...,...,...,...,...,...,...,...
99995,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casus,casusa
99996,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku
99997,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırman,tırmandığını
99998,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çöz,çözülemeyecek


In [17]:
df_lemma_all = strip(df_lemma_all)
df_lemma_all

,word,lemma.atis,lemma.boun,lemma.framenet,lemma.imst,lemma,lemma.kenet,lemma.penn,lemma.spacylookup,lemma.tourism
0,bir,bir,bir,bir,bir,bir,bir,bir,bir,bir
1,bu,bu,bu,bu,bu,bu,bu,bu,bu,bu
2,ne,ne,ne,ne,ne,ne,ne,ne,ne,ne
3,ve,ve,ve,ve,ve,ve,ve,ve,ve,ve
4,için,için,için,için,için,da,iç,da,için,için
...,...,...,...,...,...,...,...,...,...,...
99995,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casusa,casus,casusa
99996,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku,seppuku
99997,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırmandığını,tırman,tırmandığını
99998,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çözülemeyecek,çöz,çözülemeyecek


In [18]:
for column in df_lemma_all.columns:
    column_nunique = df_lemma_all[f"{column}"].nunique()
    print(f"{column} : {column_nunique}")

word : 100000
lemma.atis : 99386
lemma.boun : 86986
lemma.framenet : 96794
lemma.imst : 93539
lemma : 88593
lemma.kenet : 85433
lemma.penn : 88593
lemma.spacylookup : 28112
lemma.tourism : 100000


In [19]:
#df_lemma_all = df_lemma_all.iloc[word_start:word_end,]

In [20]:
df_lemma_select = df_lemma_all[["word","lemma.spacylookup"]]

In [21]:
df_lemma_select["lemma.spacylookup"].nunique()

28112

In [22]:
df_word_lemma_merge = pd.merge(df_word_result, df_lemma_select, how="left", on="word")
df_word_lemma_merge.drop_duplicates(inplace=True)
df_word_lemma_merge

,word,lemma.spacylookup
0,burada,bura
1,bak,bak
2,ben,ben
3,olduğunu,ol
4,biliyorum,bil
...,...,...
195,git,git
196,önce,önce
197,iyi,iyi
198,olarak,ol


In [23]:
df_word_lemma_join = df_word_lemma_merge.groupby(["lemma.spacylookup"])["word"].apply(", ".join).reset_index()
df_word_lemma_join

,lemma.spacylookup,word
0,adam,adam
1,al,al
2,ama,ama
3,anne,anne
4,artık,artık
...,...,...
142,üç,üç
143,şekil,şekilde
144,şey,"şey, şeyler, şeyi"
145,şimdi,şimdi


In [24]:
lemma_length = len(df_word_lemma_join)

In [25]:
df_word_lemma_join.to_excel(f"{word_end}_Word_{lemma_length}_Lemma_And_Join_Words.xlsx", index=False)

#### Copy Move And Delete

In [26]:
output_file = glob.glob(f"{word_end}_Word_{lemma_length}_Lemma_And_Join_Words.xlsx")
output_file

['200_Word_147_Lemma_And_Join_Words.xlsx']

In [27]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [28]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass